In [ ]:
!pip install langchain
!pip install openai
!pip install tqdm
!pip install pinecone-client
!pip install tiktoken

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone

In [ ]:
# app.py 에서 직접 수정하겠습니다.
import streamlit as st
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone

OPENAI_API_KEY = ""
PINECONE_API_KEY = ""

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

client = OpenAI(api_key=OPENAI_API_KEY)

st.set_page_config(layout="wide")

st.header("🐯슈카월드 챗봇🐯")
if "openai_model" not in st.session_state:
  st.session_state.openai_model = "gpt-3.5-turbo-0125"

if "messages" not in st.session_state:
  system_prompt = {
      "role": "system",
      "content": """당신은 경제/시사를 다루는 유튜브 채널인 슈카월드의 정보를 알려주는 챗봇입니다.
      슈카월드를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.
      경제/시사 이슈를 물어보는 사람들에게 슈카월드에서 다룬 내용을 바탕으로 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.
      당신은 사람들에게 다음과 같이 대답해야 합니다:
      - 친절한 말투
      - 항상 존댓말 사용
      - 적절한 이모지 사용
      당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."""
  }
  st.session_state.messages = [system_prompt]

if "retrievied_chunks" not in st.session_state:
  st.session_state.retrievied_chunks = []

if "summarized_query" not in st.session_state:
  st.session_state.summarized_query = ""

# 수정 전 메시지 렌더링하는 로직의 위치
# for message in st.session_state.messages:
#   if message["role"] != "system":
#     with st.chat_message(message["role"]):
#       st.markdown(message["content"])

with st.container(height=650):
  col1, col2 = st.columns([5,5], gap="medium")
  with col1:
    # 수정 후 메시지 렌더링하는 로직의 위치 (왼쪽 섹션 안으로)
    for message in st.session_state.messages:
      if message["role"] != "system":
        with st.chat_message(message["role"]):
          st.markdown(message["content"])

    st.subheader("Chat Section")
    if prompt := st.chat_input("슈카월드에서 궁금한 정보를 물어보세요😊"):
      user_prompt = {
        "role": "user",
        "content": prompt
      }
      st.session_state.messages.append(user_prompt)

      with st.chat_message("user"):
        st.markdown(prompt)

      with st.chat_message("assistant"):
        # 방식2로 Query Summarization 수행하기
        recent_query = ""
        for message in st.session_state.messages[-3:]:
          recent_query += f'{message["role"]}"\n"{message["content"]}"\n\n"'

        recent_query_prompt = {
            "role": "user",
            "content": f"2 pair of question-answers.\n{recent_query}"
        }

        summarization_system_prompt = {
            "role": "system",
            "content": "유저가 입력한 여러 질문을 파악하고 유저가 궁금한 것을 하나의 긴 문장으로 변환해줘. 대화 중간에 주제가 바뀌었으면 유저가 입력한 최신 주제에 맞춰서 정리해줘."
        }

        response = client.chat.completions.create(
          model=st.session_state.openai_model,
          messages=[summarization_system_prompt, recent_query_prompt],
          temperature=0.5,
          max_tokens=1024,
        )

        summarized_query = response.choices[0].message.content
        st.session_state.summarized_query = summarized_query

        #요약한 query를 embeddings로 변경
        response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
        query_embeddings = response.data[0].embedding

        # context 가져오기
        retrieved_chunks = index.query(
            namespace="ns3",
            vector=query_embeddings,
            top_k=7,
            include_values=False,
            include_metadata=True,
        )
        contexts = ""

        st.session_state.retrievied_chunks = []
        for match in retrieved_chunks.matches:
            context = match["metadata"]["chunk"]
            contexts += context + "\n\n"
            st.session_state.retrievied_chunks.append(context)

        context_prompt = {
            "role" : "system",
            "content" : f"[Context]\n{contexts}"
        }

        # 전체 활용하여 LLM 호출
        messages = st.session_state.messages
        messages.append(context_prompt)

        message_placeholder = st.empty()
        full_response = ""

        for response in client.chat.completions.create(
              model=st.session_state.openai_model,
              messages=messages,
              temperature=0.5,
              max_tokens=1024,
              stream=True
        ):
          full_response += (response.choices[0].delta.content or "")
          message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
      st.session_state.messages.append({"role": "assistant", "content": full_response})

    with col2:
      st.subheader("Logging Section")
      st.write("Summarized Query")
      st.write(st.session_state.summarized_query)
      st.write("Retrieved Chunks")
      for idx, chunk in enumerate(st.session_state.retrievied_chunks):
        st.write(f"{idx+1}th chunk: {chunk}")